In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')

In [7]:
df_train = pd.read_csv(path/'../../indicnlp-news-articles/pa/pa-train.csv', header=None)
df_train.head()

,0,1
0,sports,⁄ ਕ੍ਰਿਕੇਟ\n ⁄ ਜਨਰਲ\nInd vs Ban : ਚਾਰ ਸਾਲਾਂ...
1,business,ਸੈਂਸੈਕਸ ਦੀਆਂ ਟਾਪ 10 ਕੰਪਨੀਆਂ 'ਚੋਂ ਸੱਤ ਦਾ ਬਾਜ਼ਾਰ ...
2,entertainment,ਵਿੱਦਿਆ ਬਾਲਨ ਨੇ ਮਨਾਇਆ ਵਰਲਡ ਮੈਥਸ ਡੇ\nPublish Dat...
3,sports,"ਆਈਐੱਸਐੱਲ-6 ਦਾ ਆਗਾਜ਼, ਬਲਾਸਟਰਜ਼ ਨੇ ਜਿੱਤਿਆ ਮੈਚ\nPub..."
4,entertainment,ਬਚਪਨ ਦੇ ਦੋਸਤ ਲਈ ਰਣਵੀਰ ਸਿੰਘ ਨੇ ਬਦਲਿਆ ਆਪਣਾ ਸ਼ਡਿਊਲ...


In [8]:
df_valid = pd.read_csv(path/'../../indicnlp-news-articles/pa/pa-valid.csv', header=None)
df_valid.head()

,0,1
0,entertainment,⁄ ਮਨੋਰੰਜਨ\n ⁄ ਬਾਲੀਵੁੱਡ\nLady Gaga Viral V...
1,entertainment,⁄ ਮਨੋਰੰਜਨ\n ⁄ ਬਾਲੀਵੁੱਡ\nਨਸੀਰੂਦੀਨ ਸ਼ਾਹ ਦੇ ਬ...
2,sports,"ਮਾਸਟਰ ਹੈੱਡ\nPublish Date:Mon, 20 Aug 2018 10:2..."
3,business,ਟੀਵੀਐੱਸ ਮੋਟਰ ਦਾ ਮੁਨਾਫ਼ਾ ਵਧ ਕੇ ਹੋਇਆ 154.3 ਕਰੋੜ ਰ...
4,politics,⁄ ਚੋਣਾਂ\n ⁄ ਲੋਕ ਸਭਾ\nਦਰਿਆਈ ਪਾਣੀ ਹੋਇਆ ਜ਼ਹਿਰੀ...


In [9]:
df_test = pd.read_csv(path/'../../indicnlp-news-articles/pa/pa-test.csv', header=None)
df_test.head()

,0,1
0,politics,ਪ੍ਰਸ਼ਾਸਨ ਨੇ ਡਾ. ਧਰਮਵੀਰ ਗਾਂਧੀ ਦੇ ਨਾਂ 'ਤੇ ਲਾਈ ਚੇਪ...
1,sports,ਹਰ ਸਮੇਂ ਨਾਲ ਨਹੀਂ ਰਹੇਗਾ ਖਿਡਾਰੀਆਂ ਦਾ ਪਰਿਵਾਰ\nPub...
2,business,4ਜੀ ਨੂੰ ਪਛਾੜ ਐੱਲਈਡੀ ਬਲਬ ਦੇਵੇਗਾ ਹਾਈ ਸਪੀਡ ਇੰਟਰਨੈ...
3,entertainment,ਪੀਐਮ ਮੋਦੀ ਦੇ ਬਚਪਨ ’ਤੇ ਬਣੀ ਫਿ਼ਲਮ ‘ਚਲੋ ਜੀਤੇ ਹੈਂ’...
4,sports,ਬਹੁਤ ਚੰਗਾ ਮਹਿਸੂਸ ਹੋ ਰਿਹੈ : ਕੋਹਲੀ\nPublish Date...


In [10]:
df_train.shape, df_valid.shape, df_test.shape

((2496, 2), (312, 2), (312, 2))

In [11]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [12]:
label_cols = [0]

In [13]:
class PanjabiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/punjabi/tokenizer/panjabi_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [14]:
sp = spm.SentencePieceProcessor()
sp.Load('./../../models/punjabi/tokenizer/panjabi_lm.model')
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [15]:
# 30,000 is the vocab size that we chose in sentencepiece
panjabi_vocab = Vocab(itos)

In [16]:
panjabi_tok = PanjabiTokenizer('pb')

In [17]:
tokenizer = Tokenizer(tok_func=PanjabiTokenizer, lang='pb')

In [18]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [19]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=panjabi_vocab, label_cols=label_cols)

In [20]:
data_lm.show_batch()

idx,text
0,"▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ"
1,"side nt ▁ camp ▁ back s <unk> ▁s id hu ▁but ▁ che ema ▁ cri tic is ed ▁x x bo s ▁ਬੈਂਕਾਂ ▁ਦੀ ▁ਮਜ਼ਬੂਤੀ ▁ਲਈ ▁ਸਰਕਾਰ ▁ਵਚਨਬੱਧ ▁ਏਜੰਸੀ , ▁x x re p ▁7 ▁ਨਵੀਂ ▁ਦਿੱਲੀ <unk> ▁l ast ▁up d ated : <unk> ▁t u e , ▁ 03 ▁se p ▁ 2019 ▁ 03 : 35 <unk> ▁p m <unk> ▁i <unk> st ▁ਆਈ ਡੀ"
2,"ਟਾ ਲਾ ▁ਬਤੌਰ ▁ਉਪ ▁ਮੁੱਖ ▁ਮੰਤਰੀ ▁ਸਹੁੰ ▁ਚੁੱਕ ▁ਸਕਦੇ ▁ਹਨ । ▁ਅਜਿਹੇ ▁ਵਿਚ ▁ਜੇ ਜੇ ਪੀ ▁ਨਾਲ ▁ਅਜੈ ▁ਚੌ ਟਾ ਲਾ ▁ਲਈ ▁ਇਹ ▁ਇਤਿਹਾਸਕ ▁ਪਲ ਼ ▁ਹੋਵੇਗਾ । ▁post ed <unk> ▁b y : ▁see ma ▁an and ▁x x bo s ▁ ⁄ ▁ਚੋਣਾਂ ▁ ⁄ ▁ਲੋਕ ▁ਸਭਾ ▁ਮਜੀਠੀਆ ▁ਨੇ ▁ਕਿਹਾ , ' ' ਭੈਣ ▁ਜੀ ▁ਤਾਂ ▁ਨਹੀਂ , ▁ਜੇ ▁ਜੀਜਾ ▁ਜੀ ▁ਨੂੰ ▁ਚੋਣ ▁ਲੜ ਾ ▁ਲਈ ਏ ▁ਤਾਂ"
3,"▁ਸਾਰੇ ▁ਸਾਧਾਰਨ ▁ਮੁੱਦਿਆਂ ▁ਨੂੰ ▁ਭੁੱਲ ▁ਕੇ , ▁ਇਕ ▁ਦੇਸ਼ ▁ਦੇ ▁ਰੂਪ ▁' ਚ ▁ਸਾਹਮਣੇ ▁ਆ ▁ਕੇ ▁ਹੱਥ ▁ਮਿਲਾ ਈਏ ... । ▁ਵਿੱਤੀ ▁ਵਰ੍ਹੇ ▁ 2019 -20 20 ▁ਦਾ ▁ਆਗਾਜ਼ ▁ਤੇ ▁ਉਹ ▁ਵੀ ▁ਉਦੋਂ ▁ਜਦੋਂ ▁ਦੁਨੀਆ ▁ਦਾ ▁ਸਭ ▁ਤੋਂ ▁ਵੱਡਾ ▁ਲੋਕਤੰਤਰ ▁ਭਾਰਤ ▁ਆਪਣੀਆਂ ▁17 ਵੀਆਂ ▁ਲੋਕ ▁ਸਭਾ ▁ਚੋਣਾਂ ▁ਦੇ ▁ਰੂਪ ▁' ਚ ▁ਸਭ ▁ਤੋਂ ▁ਵੱਡਾ ▁ਉਤਸਵ ▁ਮਨਾਉਣ ▁ਜਾ ▁ਰਿਹਾ ▁ਹੈ , ▁ਡਾ . ▁ਕਲਾਮ ▁ਦੇ ▁ਵਿਜ਼ ਨ -20 20 ▁ਦੀ ▁ਪੂਰਤੀ"
4,▁ਆਪਣੇ ▁ਆਪ ▁' ਚ ▁ਮੁਸ਼ਕਿਲ ▁ਕੰਮ ▁ਸੀ । ▁ਖ਼ਾਸ ਕਰ ▁ਜਦੋਂ ▁ਦੇਸ਼ ▁ਨੂੰ ▁ਆਜ਼ਾਦ ▁ਹੋਏ ▁ਸਿਰਫ਼ ▁ਪੰਜ ▁ਸਾਲ ▁ਹੋਏ ▁ਹੋਣ । ▁ਇਹ ▁ਦੇਸ਼ ▁ਲਈ ▁ਪਹਿਲਾ ▁ਅਨੁਭਵ ▁ਸੀ । ▁ਇਹ ▁ਸਭ ▁ਹੋਇਆ ▁ਪਹਿਲੇ ▁ਮੁੱਖ ▁ਚੋਣ ▁ਕਮਿਸ਼ਨਰ ▁ਆਈ ਸੀ ਐੱਸ ▁( ਇੰਡੀਅਨ ▁ਸਿਵਲ ▁ਸਰਵਿਸ ) ▁ਸੁਕੁਮਾਰ ▁ਦੀ ▁ਅਗਵਾਈ ▁' ਚ । ▁ਉਹ ▁ਇਕ ▁ਗਣਿਤ ▁ਮਾਹਿਰ ▁ਵੀ ▁ਸਨ । ▁ਅਜਿਹਾ ▁ਮੰਨਿਆ ▁ਜਾਂਦਾ ▁ਹੈ ▁ਕਿ ▁ਚੋਣ ▁ਕਮਿਸ਼ਨ ▁ਦਾ ▁ਗਠਨ ▁ਤਤਕਾਲੀ ▁ਪ੍ਰਧਾਨ ▁ਮੰਤਰੀ ▁ਨਹਿਰੂ ▁ਨੇ


In [21]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, pretrained=False)

In [22]:
# Loading the pretrained language model on panjabi wikipedia
learn.load(path/'../../../models/punjabi/lm/ULMFiT/model', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (2496 items)
x: LMTextList
▁x x bo s ▁ ⁄ ▁ਕ੍ਰਿਕੇਟ ▁ ⁄ ▁ਜਨਰਲ ▁in d ▁vs <unk> ▁b an ▁: ▁ਚਾਰ ▁ਸਾਲਾਂ ▁' ਚ ▁ਖੇਡੇ ▁ਗਏ ▁ਸਿਰਫ਼ ▁11 <unk> - n ight ▁ਟੈਸਟ ▁ਮੈਚ , ▁ਭਾਰਤ ▁ਬਣੇਗਾ ▁ਅਜਿਹਾ ▁ਕਰਨ ▁ਵਾਲਾ ▁7 ਵਾਂ ▁ਦੇਸ਼ <unk> ▁p ub lish <unk> ▁d ate : t hu , ▁31 <unk> ▁o ct ▁ 2019 ▁0 9 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ ▁ਭਾਰਤ ▁ਵੀ ▁ਉਨ੍ਹਾਂ ▁ਟੀਮਾਂ ▁ਦੀ ▁ਸੂਚੀ ▁' ਚ ▁ਸ਼ਾਮਲ ▁ਹੋ ▁ਜਾਵੇਗਾ , ▁ਜਿਨ੍ਹਾਂ ▁ਨੇ ▁ਟੈਸਟ ▁ਕ੍ਰਿਕਟ ▁ਦੇ ▁ਇਸ ▁ਨਵੇਂ ▁ਰੋਮ ਾਂ ਚ ▁' ਚ ▁ਸ਼ਿਰਕ ਤ ▁ਕੀਤੀ ▁ਹੈ । ▁ਸੰਨ ▁2015 ▁' ਚ ▁ਨਵੰਬਰ ▁' ਚ ▁ਪਹਿਲੀ ▁ਵਾਰ ▁ਇੰਟਰਨੈਸ਼ਨਲ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਮੈਚ ▁ਖੇਡਿਆ ▁ਗਿਆ ▁ਸੀ ▁ਪਰ ▁ਉਦੋਂ ▁ਤੋਂ ▁ਹੁਣ ▁ਤਕ ▁ਸਿਰਫ਼ ▁11 ▁ਮੈਚ ▁ਹੀ ▁ਖੇਡੇ ▁ਜਾ ▁ਸਕੇ ▁ਹ

In [23]:
learn.freeze()

In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.242599,4.147893,0.322747,00:23


In [25]:
learn.unfreeze()

In [26]:
learn.fit_one_cycle(3, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.826468,3.969316,0.344308,00:28
1,3.642579,3.863910,0.357924,00:28
2,3.439068,3.849604,0.361077,00:28


In [27]:
learn.predict('ਜੋ ਉਹਨਾਂ ਦੇ ਰੱਬਾਂ ਨੂੰ ਪ੍ਰਸਤੁਤ ਕਰਦੇ',n_words=10)

'ਜੋ ਉਹਨਾਂ ਦੇ ਰੱਬਾਂ ਨੂੰ ਪ੍ਰਸਤੁਤ ਕਰਦੇ ▁ਹਨ ▁ਉਹ ▁ਉਨ੍ਹਾਂ ▁ਜਾ ▁ਰਹੇ ▁ਪਰ ਹੋ ▁ਹੱਥ ▁ਵੀ ▁ਕੁਝ'

In [28]:
learn.save_encoder('fine_tuned_enc')

In [29]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=panjabi_vocab, label_cols=label_cols, bs=64)

In [30]:
data_clas.show_batch()

text,target
"▁x x bo s ▁60 ▁ਸਾਲ ▁ਦਾ ▁ਪਾਪ ▁ਧੋਣ ▁ਦੀ ▁ਕੋਸ਼ਿਸ਼ ▁ਕਰ ▁ਰਹੀ ▁ਹੈ ▁ਕਾਂਗਰਸ <unk> ▁p ub lish <unk> ▁d ate : t u e , ▁0 9 <unk> ▁a pr ▁ 2019 ▁0 6 : 00 <unk> <unk> ▁( i <unk> st ) ▁ਚੋਣ ▁ਜ਼ਾਬ ਤੇ ▁ਕਾਰਨ ▁ਸਰਕਾਰ ▁ਦੇ ▁ਕੰਮਕਾਜ ▁ਰੁਕ ▁ਜਾਂਦੇ ▁ਹਨ । ▁ਦੂਜਾ ▁ਮੇਰੀ ▁ਕੋਸ਼ਿਸ਼ ▁ਹੁੰਦੀ ▁ਹੈ ▁ਕਿ ▁ਵੱਧ ▁ਤੋਂ ▁ਵੱਧ ▁ਲੋਕਾਂ ▁ਨੂੰ ▁ਮਿਲਾ । ▁ਇ",politics
"▁x x bo s ▁ਵਿਜ਼ ਨ ▁ਸਬਕ ਾ ▁ਸਾਥ - ਸਬ ਕਾ ▁ਵਿਕਾਸ ▁ 2019 -20 <unk> ▁p ub lish <unk> ▁d ate : s un , ▁31 <unk> ▁ 2019 ▁ 08 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਪਿਛਲੇ ▁ਇਕ ▁ਦਹਾਕੇ ▁' ਚ ▁ਭਾਰਤੀ ▁ਅਰਥਚਾਰਾ ▁ਕੌਮਾਂਤਰੀ ▁ਵਿਕਾਸ ▁' ਚ ▁ਸਭ ▁ਤੋਂ ▁ਵੱਡੇ ▁ਯੋਗਦਾਨ ▁ਕਰ ਤਾਵਾਂ ▁' ਚੋਂ ▁ਇਕ ▁ਰਿਹਾ ▁ਹੈ , ▁ਜੋ ▁ਆਰਥਿਕ ▁ਸਰਗਰਮੀਆਂ ▁' ਚ",politics
"▁x x bo s ▁ਸੁਪਰ ▁ਓਵਰ ▁' ਚ ▁ਜਿੱਤਿਆ ▁ਮੁ ਬੰ ਈ , ▁ਪੁੱਜਾ ▁ਪਲੇ ਆ ਫ ▁' ਚ <unk> ▁f ri , ▁ 03 ▁ <unk> <unk> ▁ 2019 ▁12 : 0 6 <unk> <unk> ▁( i <unk> st ) ▁| ▁ਹਾਈ ਲਾਈਟ ▁ਮਨੀਸ਼ ▁ਪਾਂਡੇ ▁ਨੇ ▁ਖੇਡੀ ▁71 ▁ਦੌੜਾਂ ▁ਦੀ ▁ਪਾਰੀ ▁ਵੀ ▁ਨਾ ▁ਜਿ ਤਾ ▁ਸਕੀ ▁ਹੈਦਰਾਬਾਦ ▁ਨੂੰ ▁ਹੈਦਰਾਬਾਦ ▁ਨੂੰ ▁ ਵਾਰਨਰ ▁ਦੀ ▁ਰੜਕ ੇਗੀ ▁ਘਾਟ ▁ਸੁਪਰ ▁ਓਵਰ ▁'",sports
"▁x x bo s ▁ਲੋਕਤੰਤਰ ▁ਦੇ ▁ਪਹਿਰੇਦਾਰ <unk> ▁p ub lish <unk> ▁d ate : m on , ▁15 <unk> ▁a pr ▁ 2019 ▁12 : 29 <unk> <unk> ▁( i <unk> st ) ▁ਚੋਣਾਂ ▁ਆਉਂਦੇ ▁ਹੀ ▁ਸਾਰੇ ▁ਦਲ , ▁ਵੋਟ , ▁ਚੋਣ ▁ਨਿਸ਼ਾਨ , ▁ਉਮੀਦਵਾਰ ▁ਤੇ ▁ਦੇਸ਼ ▁ਦੀਆਂ ▁ਵੱਡੀਆਂ - ਵੱਡੀਆਂ ▁ਸਿਆਸੀ ▁ਪਾਰਟੀਆਂ ▁ਦੀ ▁ਗੱਲ ▁ਕਰਦੇ ▁ਹਨ । ▁ਪਰ ▁ਇਸ ▁ਸ਼ੋਰ ▁ਵਿਚਕਾਰ ▁ਅਸੀਂ ▁ਉਸ ▁ਦੀ ▁ਭੂਮਿਕਾ ▁ਭੁੱਲ ▁ਜਾਂਦੇ ▁ਹਾਂ",politics
"▁x x bo s ▁ਬ੍ਰਾਜ਼ੀਲ ▁ਦੀ ▁ਮਾਰ ਤਾ ▁ਨੇ ▁ਸਿਰਜਿਆ ▁ਰਿਕਾਰਡ <unk> ▁p ub lish <unk> ▁d ate : f ri , ▁28 <unk> ▁j un ▁ 2019 ▁10 : 17 <unk> <unk> ▁( i <unk> st ) ▁ਫਰਾਂਸ ▁ਵਿਚ ▁ਖੇਡੇ ▁ਜਾ ▁ਰਹੇ ▁ਵਿਸ਼ਵ ▁ਫੁੱਟਬਾਲ ▁ਕੱਪ ▁' ਚ ▁ਬ੍ਰਾਜ਼ੀਲੀ ▁ਮਹਿਲਾ ▁ਟੀਮ ▁ਦੀ ▁ਕਪਤਾਨ ▁ਮਾਰ ਤਾ ▁ਨੇ ▁17 ਵਾਂ ▁ਗੋਲ ▁ਸਕੋਰ ▁ਕਰ ▁ਕੇ ▁ਇਤਿਹਾਸ ▁ਸਿਰਜਿਆ ▁ਹੈ । ▁ਬ੍ਰਾਜ਼ੀਲ ▁ਦਾ ▁ਫੁੱਟਬਾਲ ▁ਖੇਡ",sports


In [31]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)

In [32]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2496 items)
x: TextList
▁x x bo s ▁ ⁄ ▁ਕ੍ਰਿਕੇਟ ▁ ⁄ ▁ਜਨਰਲ ▁in d ▁vs <unk> ▁b an ▁: ▁ਚਾਰ ▁ਸਾਲਾਂ ▁' ਚ ▁ਖੇਡੇ ▁ਗਏ ▁ਸਿਰਫ਼ ▁11 <unk> - n ight ▁ਟੈਸਟ ▁ਮੈਚ , ▁ਭਾਰਤ ▁ਬਣੇਗਾ ▁ਅਜਿਹਾ ▁ਕਰਨ ▁ਵਾਲਾ ▁7 ਵਾਂ ▁ਦੇਸ਼ <unk> ▁p ub lish <unk> ▁d ate : t hu , ▁31 <unk> ▁o ct ▁ 2019 ▁0 9 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ ▁ਭਾਰਤ ▁ਵੀ ▁ਉਨ੍ਹਾਂ ▁ਟੀਮਾਂ ▁ਦੀ ▁ਸੂਚੀ ▁' ਚ ▁ਸ਼ਾਮਲ ▁ਹੋ ▁ਜਾਵੇਗਾ , ▁ਜਿਨ੍ਹਾਂ ▁ਨੇ ▁ਟੈਸਟ ▁ਕ੍ਰਿਕਟ ▁ਦੇ ▁ਇਸ ▁ਨਵੇਂ ▁ਰੋਮ ਾਂ ਚ ▁' ਚ ▁ਸ਼ਿਰਕ ਤ ▁ਕੀਤੀ ▁ਹੈ । ▁ਸੰਨ ▁2015 ▁' ਚ ▁ਨਵੰਬਰ ▁' ਚ ▁ਪਹਿਲੀ ▁ਵਾਰ ▁ਇੰਟਰਨੈਸ਼ਨਲ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਮੈਚ ▁ਖੇਡਿਆ ▁ਗਿਆ ▁ਸੀ ▁ਪਰ ▁ਉਦੋਂ ▁ਤੋਂ ▁ਹੁਣ ▁ਤਕ ▁ਸਿਰਫ਼ ▁11 ▁ਮੈਚ ▁ਹੀ ▁ਖੇਡੇ ▁ਜਾ ▁ਸਕੇ ▁ਹਨ । ▁

In [33]:
learn.freeze()

In [34]:
learn.loss_func.func

CrossEntropyLoss()

In [35]:
mcc = MatthewsCorreff()

In [36]:
learn.metrics = [accuracy, mcc]

In [37]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,matthews_correff,time
0,0.338626,0.209580,0.967949,0.957647,00:09


In [38]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,matthews_correff,time
0,0.178439,0.081482,0.971154,0.961679,00:11


In [39]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,accuracy,matthews_correff,time
0,0.093764,0.055583,0.977564,0.970039,00:20
1,0.073692,0.042346,0.983974,0.978509,00:20
2,0.053034,0.039399,0.983974,0.978563,00:20
3,0.036815,0.040962,0.990385,0.987149,00:20
4,0.026289,0.048591,0.980769,0.974455,00:20


Better model found at epoch 0 with accuracy value: 0.9775640964508057.
Better model found at epoch 1 with accuracy value: 0.9839743375778198.
Better model found at epoch 3 with accuracy value: 0.9903846383094788.


In [40]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2496 items)
x: TextList
▁x x bo s ▁ ⁄ ▁ਕ੍ਰਿਕੇਟ ▁ ⁄ ▁ਜਨਰਲ ▁in d ▁vs <unk> ▁b an ▁: ▁ਚਾਰ ▁ਸਾਲਾਂ ▁' ਚ ▁ਖੇਡੇ ▁ਗਏ ▁ਸਿਰਫ਼ ▁11 <unk> - n ight ▁ਟੈਸਟ ▁ਮੈਚ , ▁ਭਾਰਤ ▁ਬਣੇਗਾ ▁ਅਜਿਹਾ ▁ਕਰਨ ▁ਵਾਲਾ ▁7 ਵਾਂ ▁ਦੇਸ਼ <unk> ▁p ub lish <unk> ▁d ate : t hu , ▁31 <unk> ▁o ct ▁ 2019 ▁0 9 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ ▁ਭਾਰਤ ▁ਵੀ ▁ਉਨ੍ਹਾਂ ▁ਟੀਮਾਂ ▁ਦੀ ▁ਸੂਚੀ ▁' ਚ ▁ਸ਼ਾਮਲ ▁ਹੋ ▁ਜਾਵੇਗਾ , ▁ਜਿਨ੍ਹਾਂ ▁ਨੇ ▁ਟੈਸਟ ▁ਕ੍ਰਿਕਟ ▁ਦੇ ▁ਇਸ ▁ਨਵੇਂ ▁ਰੋਮ ਾਂ ਚ ▁' ਚ ▁ਸ਼ਿਰਕ ਤ ▁ਕੀਤੀ ▁ਹੈ । ▁ਸੰਨ ▁2015 ▁' ਚ ▁ਨਵੰਬਰ ▁' ਚ ▁ਪਹਿਲੀ ▁ਵਾਰ ▁ਇੰਟਰਨੈਸ਼ਨਲ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਮੈਚ ▁ਖੇਡਿਆ ▁ਗਿਆ ▁ਸੀ ▁ਪਰ ▁ਉਦੋਂ ▁ਤੋਂ ▁ਹੁਣ ▁ਤਕ ▁ਸਿਰਫ਼ ▁11 ▁ਮੈਚ ▁ਹੀ ▁ਖੇਡੇ ▁ਜਾ ▁ਸਕੇ ▁ਹਨ । ▁

In [41]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,entertainment,business,sports,politics
0,ਪ੍ਰਸ਼ਾਸਨ ਨੇ ਡਾ. ਧਰਮਵੀਰ ਗਾਂਧੀ ਦੇ ਨਾਂ 'ਤੇ ਲਾਈ ਚੇਪ...,politics,politics,0.000129951,0.000259213,0.00466655,0.994944
1,ਹਰ ਸਮੇਂ ਨਾਲ ਨਹੀਂ ਰਹੇਗਾ ਖਿਡਾਰੀਆਂ ਦਾ ਪਰਿਵਾਰ\nPub...,sports,sports,1.25987e-05,5.31181e-05,0.999915,1.89882e-05
2,4ਜੀ ਨੂੰ ਪਛਾੜ ਐੱਲਈਡੀ ਬਲਬ ਦੇਵੇਗਾ ਹਾਈ ਸਪੀਡ ਇੰਟਰਨੈ...,business,business,0.000964326,0.980661,0.0173535,0.00102142
3,ਪੀਐਮ ਮੋਦੀ ਦੇ ਬਚਪਨ ’ਤੇ ਬਣੀ ਫਿ਼ਲਮ ‘ਚਲੋ ਜੀਤੇ ਹੈਂ’...,entertainment,entertainment,0.998856,0.000165984,0.00024856,0.000729822
4,ਬਹੁਤ ਚੰਗਾ ਮਹਿਸੂਸ ਹੋ ਰਿਹੈ : ਕੋਹਲੀ\nPublish Date...,sports,sports,0.00115954,0.000568325,0.998111,0.000161507


In [42]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.9711538461538461

In [43]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.9616754847237609

In [44]:
df_result.to_csv('indicnlp_news_articles_pa_result.csv', index=False)